In [ ]:
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore",category=UserWarning)
warnings.simplefilter(action="ignore",category=FutureWarning)

# Suppress valuewarning when fitting ARIMA model.
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore', ValueWarning)


# Interactive plots embedded within the notebook
#%matplotlib notebook 
# Static images of plots embedded within the notebook  
%config InlineBackend.figure_formats = {'png', 'retina'}

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from platform import python_version
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels as stm
import sklearn as skl


print('Python version', python_version())
print('Numpy version', np.__version__)
print('Scipy version', sp.__version__)
print('Pandas version', pd.__version__)
print('Matplotlib version', mpl.__version__)
print('Seaborn version', sns.__version__)
print('Statsmodels version', stm.__version__)
print('Scikit-learn version', skl.__version__)

# Decision Tree 

## Load data

In [2]:
import pandas as pd
enroll_df = pd.read_excel('data/supervised-learning.xlsx', sheet_name='ENROLL')

<font color='blue'>Look at the first few rows

<font color='blue'>Inspect basic information of dataset

<font color='blue'> Look at unique values of categorical columns to determine appropriate encoding


## Data preprocessing

In [ ]:

# Rename columns
enroll_df.rename({'Jobsatisfaction': 'JobSat', 'Enrolls':'Target'}, axis=1, inplace=True)
enroll_df

In [4]:
df = enroll_df.copy()

<font color='blue'>One-hot encoding of columns `JobSat` and `Desire`

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc_onehot = OneHotEncoder(handle_unknown='ignore', 
                           drop='if_binary', dtype=int)
onehot_columns = ['JobSat','Desire']

onehot_data = enc_onehot.fit_transform(df[onehot_columns])

onehot_df = pd.DataFrame(onehot_data.toarray(), 
                         columns=enc_onehot.get_feature_names_out())
onehot_df


Concatenate one-hot encoding columns to the input dataframe

In [ ]:
df = df.drop(onehot_columns, axis=1); 
df = pd.concat([df, onehot_df], axis=1)
df

Ordinal encoding of columns `Age`, `Income`, `Target`

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

ord_columns = ['Age', 'Income']

# Specify the label order to encode 
Age_cat = ['<=30', '31 to 40', '>40']
Income_cat = ['Low', 'Medium', 'High']

enc_ord = OrdinalEncoder(categories=[Age_cat, Income_cat], dtype=np.int64)

df[ord_columns] = enc_ord.fit_transform(df[ord_columns])
df

Separate feature set and target variable

In [ ]:
X = df.drop('Target', axis=1)
Y = df[['Target']]
X
Y

## Model fitting and prediction

<font color='blue'>Fit the decision tree model to the train data

### Use the fitted model to predict

In [ ]:
# See the features used in the model and their label 
dt_clf.feature_names_in_
enc_ord.categories_
enc_onehot.categories_

Use the encoders created earlier to construct the input for the model

In [ ]:
# Encode the input {Age > 40, Income='Medium', JobSat='No', Desire='Excellent'} 
Xnew = enc_ord.transform([['>40', 'Medium']]) 
Xnew = np.hstack((Xnew, enc_onehot.transform([['No', 'Excellent']]).toarray()))
Xnew

Predict the target label and its probability

In [ ]:
# Make the prediction 
dt_clf.predict(Xnew) 
dt_clf.predict_proba(Xnew)

## Evaluate performance

<font color='blue'>Determine the classification report for the train data

## Visualizing rules and tree

In [ ]:
dt_clf.feature_names_in_

In [ ]:
from sklearn import tree

text_representation = tree.export_text(dt_clf, 
                                       feature_names=dt_clf.feature_names_in_.tolist())
print(text_representation)

(Optional) Install necessary packages by running the following commands in the terminal:

`conda install python-graphviz`  
`pip install pydotplus`

In [ ]:
import graphviz
from sklearn import tree

dot_data = tree.export_graphviz(dt_clf, out_file=None,
                           feature_names=X.columns,
                           class_names=enroll_df['Target'].unique(), impurity=False,
                           filled=True, rounded=True, proportion=True)
# Draw graph
graph = graphviz.Source(dot_data, format='png')  
graph

# graph = graphviz.Source(dot_data)  
# graph.render('enroll', format='png', view=True)

## Feature Importance

<font color='blue'>Create a list of feature importance and sort.

In [ ]:
feature_list = pd.DataFrame({'feature':X.columns, 'value':dt_clf.feature_importances_})
feature_list_sorted = feature_list.sort_values('value')
feature_list_sorted

Plot the feature importance

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,3))
plt.barh(range(0,len(feature_list_sorted.index)), 
         feature_list_sorted.value, tick_label=feature_list_sorted.feature)
plt.tight_layout();